In [4]:
import numpy as np

In [29]:
# angstroem, deg, kcal/mol, charge
# type, element, bond, angle, distance, energy, scale, charge
d_uff = np.dtype([
    ("type", np.unicode_, 8), 
    ("element", np.unicode_, 2), 
    ("bond", "d"),
    ("angle", "d"),
    ("distance", "d"),
    ("energy", "d"),
    ("scale", "d"),
    ("charge", "d"),
])
uff_data = np.loadtxt("uff.txt", dtype=d_uff)
#print(uff_data[0]["type"])

In [30]:
mass_dict = {}
with open("masses.txt") as f:
    for line in f.readlines():
        elem, mass = line.split()
        mass_dict[elem] = mass
#print(mass_dict)

In [33]:
with open("uff.xml", "w") as f:
    f.write("<ForceField>\n <AtomTypes>\n")
    for atom in uff_data:
        try:
            f.write(
                '  <Type name="{0}" class="{0}" element="{1}" mass="{2}" def="" desc=""/>\n'.format(
                    atom["type"], atom["element"], mass_dict[atom["element"]]
                )
            )
        except KeyError:
            pass # if we don't have the mass, skip it for now
    f.write(' </AtomTypes>\n <NonbondedForce coulomb14scale="0.833333" lj14scale="0.5">')
    for atom in uff_data:
        try:
            mass_dict[atom["element"]]
            f.write(
                '  <Atom type="{0}" charge="{1}" sigma="{2}" epsilon="{3}"/>\n'.format(
                    atom["type"], atom["charge"], atom["distance"], atom["energy"]
                )
            )
        except KeyError:
            pass
    f.write(' </NonbondedForce>\n</ForceField>')

In [1]:
!cat forcefield_template.xml

<ForceField>
 <AtomTypes>
  <Type name="opls_135" class="CT" element="C" mass="12.01100" def="[C;X4](H)(H)(H)C" desc="alkane CH3"/>
  <Type name="opls_136" class="CT" element="C" mass="12.01100" def="O" desc="alkane CH2"/>
  <Type name="opls_137" class="CT" element="C" mass="12.01100" def="O" desc="alkane CH"/>
  <Type name="opls_138" class="CT" element="C" mass="12.01100" def="O" desc="alkane CH4"/>
  <Type name="opls_139" class="CT" element="C" mass="12.01100" def="O" desc="alkane C"/>
  <Type name="opls_140" class="HC" element="H" mass="1.00800"  def="H[C;X4]" desc="alkane H"/>
  <Type name="opls_142" class="CM" element="C" mass="12.01100" def="[C;X3](H)(C)C" desc="alkene C (RH-C=)"/>
  <Type name="opls_143" class="CM" element="C" mass="12.01100" def="O" desc="alkene C (H2-C=)"/>
  <Type name="opls_144" class="HC" element="H" mass="1.00800"  def="H[C;X3]" desc="alkene H"/>
  <Type name="opls_145" class="CA" element="C" mass="12.01100" def="O" desc="benzene C"/>
  <Type name="opls_14